<a href="https://colab.research.google.com/github/KashyapCKotak/AI-ML-experiments/blob/master/SeeInDark_Keras_Tensorflow_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import requests
# import os

# def download_file_from_google_drive(id, destination):
#     URL = "https://docs.google.com/uc?export=download"

#     session = requests.Session()

#     response = session.get(URL, params = { 'id' : id }, stream = True)
#     token = get_confirm_token(response)

#     if token:
#         params = { 'id' : id, 'confirm' : token }
#         response = session.get(URL, params = params, stream = True)

#     save_response_content(response, destination)    

# def get_confirm_token(response):
#     for key, value in response.cookies.items():
#         if key.startswith('download_warning'):
#             return value

#     return None

# def save_response_content(response, destination):
#     CHUNK_SIZE = 32768

#     with open(destination, "wb") as f:
#         for chunk in response.iter_content(CHUNK_SIZE):
#             if chunk: # filter out keep-alive new chunks
#                 f.write(chunk)



# print('Dowloading Sony subset... (25GB)')
# download_file_from_google_drive('10kpAcvldtcb9G2ze5hTcF1odzu4V_Zvh', './Sony.zip')

# #print('Dowloading Fuji subset... (52GB)')
# #download_file_from_google_drive('12hvKCjwuilKTZPe9EZ7ZTb-azOmUA3HT', 'dataset/Fuji.zip')

# os.system('unzip ./Sony.zip -d dataset')
# #os.system('unzip dataset/Fuji.zip -d dataset')


In [0]:
# import shutil
# shutil.rmtree('result_Sony', ignore_errors=True, onerror=None)

In [0]:
# os.system('unzip ./Sony.zip -d dataset')
#os.system('unzip dataset/Fuji.zip -d dataset')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import json
import glob
import os
epoch_file='/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/save_Sony/epochNum.txt'
save_dir='/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/save_Sony/'
last_epoch=0
last_line=""
with open(epoch_file, mode='r', buffering=1) as epoch_file:
  last_line=list(epoch_file)[-1]
last_epoch=json.loads(last_line)['epoch']
print("Epoch to start:",last_epoch)
list_of_files=glob.glob(save_dir+"*.hdf5")
if not last_epoch<10:
  latest_file_path = max(list_of_files, key=os.path.getctime)
  print ("Latest file path:",latest_file_path)

In [0]:
# with open('/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/save_Sony/epochNum.txt', mode='w', buffering=1) as epoch_file:
#   epoch_file.write(json.dumps({'epoch': -1, 'loss': 0, 'datetime': 'dummy start line'}) + '\n'),

**Main Train Code:**
=================================

In [0]:
!pip install rawpy

In [0]:
from __future__ import division
import os, time, scipy.io
import tensorflow as tf
import numpy as np
import rawpy
import glob
import keras
import shutil
import datetime as dt
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler
import threading
import json

In [0]:
# !pip install -q tf-nightly-2.0-preview
# # Load the TensorBoard notebook extension
# %load_ext tensorboard

In [0]:
# import shutil
# shutil.rmtree('result_Sony', ignore_errors=True, onerror=None)

# input_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/short/'
# gt_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/long/'
# os.mkdir('result_Sony')
# checkpoint_dir = './result_Sony/'
# result_dir = './result_Sony/'

In [0]:
def lrelu(x): # not used. Used Kears LeakyReLU with alpha 0.2 instead
    return tf.maximum(x * 0.2, x)

In [0]:
""" This cell contains the custom Lambda layers to be used in building the  keras mdoel """
def upsample_and_concat(x1,x2,output_channels,in_channels,layer,verbose=False):
  if(verbose):
    print("layer:",layer)
  pool_size = 2
  deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
  if(verbose):
    print("x1 shape:",x1.shape)
    print("x2 shape:",x2.shape)
  deconvtf=tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])
  if(verbose):
    print("deconvtf shape:",deconvtf.shape)
  #deconv=keras.layers.Conv2DTranspose(filters=output_channels,kernel_size=(pool_size, pool_size),strides=(pool_size, pool_size))(x1)
  #print("deconvk shape:",deconv.shape)

  deconv_output = tf.concat([deconvtf, x2], 3)
  if(verbose):
    print("deconv_output shape:",deconv_output.shape)
  deconv_output.set_shape([None, None, None, output_channels * 2])

  return deconv_output

def Depth_to_space_tf(input):
  return tf.depth_to_space(input, 2)

In [0]:
# testing code
# t=tf.ones([1,450,350,4])
# deconv_filter = tf.Variable(tf.truncated_normal([2, 2, 256, 512], stddev=0.02))
# tf.nn.conv2d_transpose(t, deconv_filter, [1,450,350,], strides=[1, pool_size, pool_size, 1])

In [0]:
def build_model():
  inputs=keras.layers.Input(shape=(None, None, 4))
  #definition: slim.conv2d(inputs,number of outputs, kernel shape)
  #definition: slim.max_pool2d(inputsconv2d_transpose ,kernel_size, padding)
  #conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
  conv1=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',name='conv1_1')(inputs)
  conv1=keras.layers.LeakyReLU(alpha=0.2,name='conv1_1_relu')(conv1)
  print("conv1 shape:",conv1.shape)
  #conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
  conv1=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',name='conv1_2')(conv1)
  conv1=keras.layers.LeakyReLU(alpha=0.2,name='conv1_2_relu')(conv1)
  print("conv1 shape:",conv1.shape)
  #pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')
  pool1=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool1')(conv1)
  print("conv1 shape after pool:",pool1.shape)

  #conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
  conv2=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',name='conv2_1')(pool1)
  conv2=keras.layers.LeakyReLU(alpha=0.2,name='conv2_1_relu')(conv2)
  #conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
  conv2=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',name='conv2_2')(conv2)
  conv2=keras.layers.LeakyReLU(alpha=0.2,name='conv2_2_relu')(conv2)
  #pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')
  pool2=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool2')(conv2)

  #conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
  conv3=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',name='conv3_1')(pool2)
  conv3=keras.layers.LeakyReLU(alpha=0.2,name='conv3_1_relu')(conv3)
  #conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
  conv3=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',name='conv3_2')(conv3)
  conv3=keras.layers.LeakyReLU(alpha=0.2,name='conv3_2_relu')(conv3)
  #pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')
  pool3=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool3')(conv3)

  #conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
  conv4=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',name='conv4_1')(pool3)
  conv4=keras.layers.LeakyReLU(alpha=0.2,name='conv4_1_relu')(conv4)
  #conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
  conv4=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',name='conv4_2')(conv4)
  conv4=keras.layers.LeakyReLU(alpha=0.2,name='conv4_2_relu')(conv4)
  #pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')
  pool4=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool4')(conv4)

  #conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
  conv5=keras.layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',name='conv5_1')(pool4)
  conv5=keras.layers.LeakyReLU(alpha=0.2,name='conv5_1_relu')(conv5)
  #conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')
  conv5=keras.layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',name='conv5_2')(conv5)
  conv5=keras.layers.LeakyReLU(alpha=0.2,name='conv5_2_relu')(conv5)

  #up6 = upsample_and_concat(conv5, conv4, 256, 512)
  up6=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv4,'output_channels':256,'in_channels':512,'layer':'upsample_concat_1','verbose':False},name='upsample_concat_1')(conv5)
  #conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
  conv6=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',name='conv6_1')(up6)
  conv6=keras.layers.LeakyReLU(alpha=0.2,name='conv6_1_relu')(conv6)
  #conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')
  conv6=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',name='conv6_2')(conv6)
  conv6=keras.layers.LeakyReLU(alpha=0.2,name='conv6_2_relu')(conv6)

  #up7 = upsample_and_concat(conv6, conv3, 128, 256)
  up7=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv3,'output_channels':128,'in_channels':256,'layer':'upsample_concat_2','verbose':False},name='upsample_concat_2')(conv6)
  #conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
  conv7=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',name='conv7_1')(up7)
  conv7=keras.layers.LeakyReLU(alpha=0.2,name='conv7_1_relu')(conv7)
  #conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')
  conv7=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',name='conv7_2')(conv7)
  conv7=keras.layers.LeakyReLU(alpha=0.2,name='conv7_2_relu')(conv7)

  #up8 = upsample_and_concat(conv7, conv2, 64, 128)
  up8=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv2,'output_channels':64,'in_channels':128,'layer':'upsample_concat_3','verbose':False},name='upsample_concat_3')(conv7)
  #conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
  conv8=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',name='conv8_1')(up8)
  conv8=keras.layers.LeakyReLU(alpha=0.2,name='conv8_1_relu')(conv8)
  #conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')
  conv8=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',name='conv8_2')(conv8)
  conv8=keras.layers.LeakyReLU(alpha=0.2,name='conv8_2_relu')(conv8)

  #up9 = upsample_and_concat(conv8, conv1, 32, 64)
  up9=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv1,'output_channels':32,'in_channels':64,'layer':'upsample_concat_4','verbose':False},name='upsample_concat_4')(conv8)
  #conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
  conv9=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',name='conv9_1')(up9)
  conv9=keras.layers.LeakyReLU(alpha=0.2,name='conv9_1_relu')(conv9)
  #conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')
  conv9=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',name='conv9_2')(conv9)
  conv9=keras.layers.LeakyReLU(alpha=0.2,name='conv9_2_relu')(conv9)
  print("conv9 shape:",conv9.shape)

  #conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
  conv10=keras.layers.Conv2D(filters=12,kernel_size=(1,1),strides=(1,1),name='conv10')(conv9)
  print("conv10 shape:",conv10.shape)
  #no activation function for this last layer

  predictions = keras.layers.core.Lambda(Depth_to_space_tf,name='depth_to_space')(conv10)
  model = keras.models.Model(inputs=inputs, outputs=predictions)
  
  return model

In [0]:
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    #print("im shape:",im.shape)
    #print("im:")
    #print(im)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level and normalise with 16383 
    ##
    # Seems 16383 is the max value due to a 14 bit depth. can be replaced with 
    ##
    #print("after normalisation")
    #print(im)

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]
    #print("after expand dims:%s ,H:%d ,W:%d " % (img_shape,H,W))

    ###
    # single bayer pixel format:
    # 0:R 1:G
    # 2:G 3:B
    ###
    out = np.concatenate((im[0:H:2, 0:W:2, :], # get the 0th Red as per above format
                          im[0:H:2, 1:W:2, :], # get the 1st Green
                          im[1:H:2, 1:W:2, :], # get the 2nd Green
                          im[1:H:2, 0:W:2, :]), axis=2) # get the 3rd Green
    #print("out.shape:",out.shape)
    #print(out)
    return out


In [0]:
class Params():
  def __init__(self):
    shutil.rmtree('/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/result_Sony', ignore_errors=True, onerror=None)
    self.input_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/short/'
    self.gt_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/long/'
    os.mkdir('/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/result_Sony')
    self.checkpoint_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/result_Sony/'
    self.result_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/result_Sony/'
    self.save_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/save_Sony/'
    self.epoch_file = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/save_Sony/epochNum.txt'
    self.tensorboard_log_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/tensorboard'
    if os.path.exists(self.tensorboard_log_dir):
      pass
    else:
      os.mkdir(self.tensorboard_log_dir)
    if os.path.exists(self.save_dir):
      pass
    else:
      os.mkdir('/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/save_Sony')
    
    
    self.epochs=4000
    self.epoch_counter=0
    self.ps=512 # batch/patch size for images for training
    self.learning_rate=0.0001 # not used in keras

In [0]:
class Data():
  def __init__(self,params):
    self.train_fns = glob.glob(params.gt_dir + '0*.ARW')
    """ 0* is for Training images. 1 for Test and 2 for Validation """
    self.train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in self.train_fns]
    print("Total Train Ids:",len(self.train_ids))
    print("Sample Train Ids:",self.train_ids[0:10])
    self.total_train_ids=len(self.train_ids)
    
    self.valid_fns = glob.glob(params.gt_dir + '2*.ARW')
    """ 0* is for Training images. 1 for Test and 2 for Validation """
    self.valid_ids = [int(os.path.basename(valid_fn)[0:5]) for valid_fn in self.valid_fns]
    print("Total Validation Ids:",len(self.valid_ids))
    print("Sample Validation Ids:",self.valid_ids[0:10])
    self.total_valid_ids=len(self.valid_ids)

In [0]:
class DataLoader(keras.utils.Sequence):
  """Class to continously load images"""
  
  def __init__(self,params,data):
    self.epochs=params.epochs
    self.input_dir=params.input_dir
    self.gt_dir=params.gt_dir
    self.train_ids=data.train_ids
    self.gt_image_map=[None] * 6000
    self.shuffled_ids=np.random.permutation(len(self.train_ids))
#     for index, val in np.ndenumerate(self.shuffled_ids):
#       print ('index:{}, image:{}'.format(index[0], val))
    self.epoch_counter=0
    # self.ind=-1
    self.ps=params.ps # batch/patch size for images for training
    #self.lock = threading.Lock() # used for making thread safe iterators
    #self.on_epoch_end()
    
#     self.gt_images = [None] * 6000
#     self.input_images = {}
#     self.input_images['300'] = [None] * len(train_ids)
#     self.input_images['250'] = [None] * len(train_ids)
#     self.input_images['100'] = [None] * len(train_ids)
    
  def on_epoch_end(self):
    self.shuffled_ids=np.random.permutation(len(self.train_ids))
#     print("in on epoch end")

  def loadImages(self):
    i=0
    for train_id in self.train_ids:
      if(i>=200):
        self.gt_image_map[train_id]=None
      else:
        print('loading gt_image id:',train_id)
        gt_files = glob.glob(self.gt_dir + '%05d_00*.ARW' % train_id) # get the ground truth image path(s) (only 1 may exist. hence select [0]th below)
        gt_path = gt_files[0] # get the first one (as only 1 gt image should exist)
        gt_raw = rawpy.imread(gt_path)
        im = gt_raw.postprocess(use_camera_wb=True,
                      half_size=False,
                      no_auto_bright=True, output_bps=16)
        gt_images = np.expand_dims(np.float32(im / 65535.0),axis=0) # divide by 65535 to normalise (scale between 0 and 1)
        self.gt_image_map[train_id]=gt_images
        i+=1      
    print('gt_images loading done')

  def __len__(self):
    'Denotes the number of batches per epoch'
    return len(self.train_ids)

  def __getitem__(self, ind):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    train_id = self.train_ids[self.shuffled_ids[ind]]
    #print("retireved the train id: {}, ind: {}".format(train_id,ind))
    in_files = glob.glob(self.input_dir + '%05d_00*.ARW' % train_id) # get all the images path with pattern 0*train_id
    try:
      in_path = in_files[np.random.randint(0, len(in_files))] # get any one image randomly
    except:
      print("exception- retireved the train id:{}, ind:{}, in_files:{}".format(train_id,ind,in_files))

    in_fn = os.path.basename(in_path) # get only the full image name

    gt_files = glob.glob(self.gt_dir + '%05d_00*.ARW' % train_id) # get the ground truth image path(s) (only 1 may exist. hence select [0]th below)
    gt_path = gt_files[0] # get the first one (as only 1 gt image should exist)
    gt_fn = os.path.basename(gt_path) # get only the full image name for gt image
    in_exposure = float(in_fn[9:-5]) # get the exposure for input image
    gt_exposure = float(gt_fn[9:-5]) # get the exposure for gt image
    ratio = min(gt_exposure / in_exposure, 300) # get the amplification ratio

    start = time.process_time()
    #cnt += 1

    #if input_images[str(ratio)[0:3]][ind] is None: # if image is not loaded (first epoch), load it
    raw = rawpy.imread(in_path)
    input_images = np.expand_dims(pack_raw(raw), axis=0) * ratio # pack the bayer image in 4 channels of RGBG
    if self.gt_image_map[train_id] is None:
      print("reading gt image")
      # print("dict len now:",len(self.gt_image_map))
      gt_raw = rawpy.imread(gt_path)
      im = gt_raw.postprocess(use_camera_wb=True,
                    half_size=False,
                    no_auto_bright=True, output_bps=16)
      gt_images = np.expand_dims(np.float32(im / 65535.0),axis=0) # divide by 65535 to normalise (scale between 0 and 1)
      self.gt_image_map[train_id]=gt_images
    else:
      gt_images = self.gt_image_map[train_id]
    # crop
#     print("time taken for reading image %d:%s with in_path: %s & gt_path:%s" % (train_id,(time.process_time() - start),in_path,gt_path))

    H = input_images.shape[1] # get the image height (number of rows)
    W = input_images.shape[2] # get the image width (number of columns)

    xx = np.random.randint(0, W - self.ps) # get a random number in W-ps (W-512)
    yy = np.random.randint(0, H - self.ps) # get a random number in H-ps (H-512)
    input_patch = input_images[:, yy:yy + self.ps, xx:xx + self.ps, :]
    gt_patch = gt_images[:, yy * 2:yy * 2 + self.ps * 2, xx * 2:xx * 2 + self.ps * 2, :]

    if np.random.randint(2) == 1:  # random flip for rows
      input_patch = np.flip(input_patch, axis=1)
      gt_patch = np.flip(gt_patch, axis=1)
    if np.random.randint(2) == 1:  # random flip for columns
      input_patch = np.flip(input_patch, axis=2)
      gt_patch = np.flip(gt_patch, axis=2)
    if np.random.randint(2) == 1:  # random transpose
      input_patch = np.transpose(input_patch, (0, 2, 1, 3))
      gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))\

    input_patch = np.minimum(input_patch, 1.0)
    # print('generated an image with ind:',ind)

    return (input_patch,gt_patch)
    

In [0]:
# import datetime
# class StoreEpochCallBack(tf.keras.callbacks.Callback):
#   def __init__(self, params):
#     self.f=open(params.epoch_num_file,"w")
#   def on_train_batch_begin(self, batch, logs=None):
#     print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

#   def on_train_batch_end(self, batch, logs=None):
#     print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

#   def on_test_batch_begin(self, batch, logs=None):
#     print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

#   def on_test_batch_end(self, batch, logs=None):
#     print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))
def learningratescheduler(epoch,lr):
  if (epoch<50):
    return 0.0005
  elif (epoch>=50 and epoch<100):
    return 0.0003
  elif (epoch>=100 and epoch<1000):
    return 0.0002
  elif (epoch>=1000 and epoch<2000):
    return 0.0001
  else:
    return 0.00001
  

def train_generator(model,data_gen,epochs,steps_per_epoch,save_fname,epoch_num_file,save_dir,tensorboard_log_dir):
  print("Model Training started!!! Jay Yogeshwar!!!")
  last_epoch=0
  last_line=""
  with open(epoch_num_file, mode='r', buffering=1) as begin_epoch_file:
    last_line=list(begin_epoch_file)[-1]
  last_epoch=json.loads(last_line)['epoch']
  print("Epoch to start:",last_epoch)
  if(last_epoch>10):
    list_of_files=glob.glob(save_dir+"*.hdf5")
    latest_file_path = max(list_of_files, key=os.path.getctime)
    print ("Latest saved model path:",latest_file_path)
    model.load_weights(latest_file_path)
  else:
    last_epoch=0
  epoch_file=open(epoch_num_file, mode='a', buffering=1)
  epoch_callback = keras.callbacks.LambdaCallback(
    on_epoch_begin=lambda epoch, logs: epoch_file.write(
        json.dumps({'epoch': (epoch+1), 'loss': logs['loss'], 'datetime': (dt.datetime.now().strftime('%d%m%Y-%H%M%S'))}) + '\n'),
    on_train_end=lambda logs: epoch_file.close()
  )
  callbacks = [
    epoch_callback,
    ModelCheckpoint(period=1,filepath=save_fname, mode='min', monitor='loss', save_weights_only=True, verbose=1, save_best_only=True),
    TensorBoard(log_dir=tensorboard_log_dir),
    LearningRateScheduler(learningratescheduler,verbose=1)
  ]
  model_history=model.fit_generator(
      initial_epoch=last_epoch,
      generator=data_gen,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      callbacks=callbacks,
      max_queue_size=5,
      workers=2,
      use_multiprocessing=True
  )
  print("Model Training completed!!! Jay Yogeshwar!!!")
  return model_history

In [0]:
from keras import backend as K

def recall(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def reduced_mean(y_true,y_pred):
  return tf.reduce_mean(tf.abs(y_pred - y_true))


params = Params()
data = Data(params)
dataloader = DataLoader(params,data)
model=build_model()
with open('model_summary.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
keras.utils.vis_utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.compile(optimizer=keras.optimizers.Adam(params.learning_rate),loss=reduced_mean,metrics=['accuracy'])
steps_per_epoch=len(data.train_ids)
# save_fname = os.path.join(params.save_dir, '%s-e%s.h5' % (dt.datetime.now().strftime('%d%m%Y-%H%M%S'), str(epochs)))
save_fname = os.path.join(params.save_dir, '%s-weights-e-{epoch:02d}-l-{loss:.2f}.hdf5' % (dt.datetime.now().strftime('%d%m%Y-%H%M%S')))



In [0]:
dataloader.loadImages()

In [0]:
""" ================ TRAIN THE MODEL ================ """
# steps_per_epoch=10
model_history=train_generator(model,dataloader,params.epochs,steps_per_epoch,save_fname,params.epoch_file,params.save_dir,params.tensorboard_log_dir)

In [0]:
# for epoch in range(lastepoch, 4001):
#     #if os.path.isdir('result/%04d' % epoch):
#     #    continue
#     ###
#     # used for skipping the current epoch if the checkpoint directory exists
#     ###
#     cnt = 0
#     if epoch > 2000:
#         learning_rate = 1e-5

#     for ind in np.random.permutation(len(train_ids)):

#         # get the path from image id

#         train_id = train_ids[ind]
#         in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id) # get all the images path with pattern 0*train_id
#         in_path = in_files[np.random.randint(0, len(in_files))] # get any one image randomly
#         in_fn = os.path.basename(in_path) # get only the full image name

#         gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id) # get the ground truth image path(s) (only 1 may exist. hence select [0]th below)
#         gt_path = gt_files[0] # get the first one (as only 1 gt image should exist)
#         gt_fn = os.path.basename(gt_path) # get only the full image name for gt image
#         in_exposure = float(in_fn[9:-5]) # get the exposure for input image
#         gt_exposure = float(gt_fn[9:-5]) # get the exposure for gt image
#         ratio = min(gt_exposure / in_exposure, 300) # get the amplification ratio

#         st = time.time()
#         cnt += 1

#         if input_images[str(ratio)[0:3]][ind] is None: # if image is not loaded (first epoch), load it
#             raw = rawpy.imread(in_path)
#             input_images[str(ratio)[0:3]][ind] = \
#                 np.expand_dims(pack_raw(raw), axis=0) * ratio # pack the bayer image in 4 channels of RGBG

#             gt_raw = rawpy.imread(gt_path)
#             im = gt_raw.postprocess(use_camera_wb=True,
#                                     half_size=False,
#                                     no_auto_bright=True, output_bps=16)
#             gt_images[ind] = np.expand_dims(np.float32(im / 65535.0),axis=0) # divide by 65535 to normalise (scale between 0 and 1)

#         # crop

#         H = input_images[str(ratio)[0:3]][ind].shape[1] # get the image height (number of rows)
#         W = input_images[str(ratio)[0:3]][ind].shape[2] # get the image width (number of columns)

#         xx = np.random.randint(0, W - ps) # get a random number in W-ps (W-512)
#         yy = np.random.randint(0, H - ps) # get a random number in H-ps (H-512)
#         input_patch = input_images[str(ratio)[0:3]][ind][:, yy:yy + ps,
#                 xx:xx + ps, :]
#         gt_patch = gt_images[ind][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx
#                                   * 2 + ps * 2, :]

#         if np.random.randint(2) == 1:  # random flip for rows
#             input_patch = np.flip(input_patch, axis=1)
#             gt_patch = np.flip(gt_patch, axis=1)
#         if np.random.randint(2) == 1:  # random flip for columns
#             input_patch = np.flip(input_patch, axis=2)
#             gt_patch = np.flip(gt_patch, axis=2)
#         if np.random.randint(2) == 1:  # random transpose
#             input_patch = np.transpose(input_patch, (0, 2, 1, 3))
#             gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))\

#         input_patch = np.minimum(input_patch, 1.0)

#         (_, G_current, output) = sess.run([G_opt, G_loss, out_image],
#                 feed_dict={in_image: input_patch, gt_image: gt_patch,
#                 lr: learning_rate})
#         output = np.minimum(np.maximum(output, 0), 1)
#         g_loss[ind] = G_current

#         print '%d %d Loss=%.3f Time=%.3f' % (epoch, cnt,
#                 np.mean(g_loss[np.where(g_loss)]), time.time() - st)

#         if epoch % save_freq == 0:
#             if not os.path.isdir(result_dir + '%04d' % epoch):
#                 os.makedirs(result_dir + '%04d' % epoch)

#             temp = np.concatenate((gt_patch[0, :, :, :], output[0, :, :
#                                   , :]), axis=1)
#             scipy.misc.toimage(temp * 255, high=255, low=0, cmin=0,
#                                cmax=255).save(result_dir
#                     + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id,
#                     ratio))

#     saver.save(sess, checkpoint_dir + 'model.ckpt')

In [0]:
import multiprocessing

multiprocessing.gpu_count()

In [0]:
# %tensorboard --logdir '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/tensorboard'
# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances
# notebook.display(port=6006, height=1000) 